In [1]:
%load_ext autoreload
%env SPARK_HOME=/usr/hdp/current/spark2-client

import findspark
findspark.init()

print('findspark initialized ...')

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, column, max, min

spark = SparkSession.builder.appName('mlonspark')\
    .config('spark.jars', '/opt/dev/target/ml-on-spark-1.0.jar')\
    .getOrCreate()

print('pyspark ready ...')

env: SPARK_HOME=/usr/hdp/current/spark2-client
findspark initialized ...
pyspark ready ...


In [2]:
train = spark.read.format("parquet").load("/data/books/ratings-train.parquet")
train.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)



In [3]:
%autoreload
from mlonspark.alternating_least_square import AlternatingLeastSquare

als = AlternatingLeastSquare()\
    .setUserCol("userId")\
    .setItemCol("bookId")\
    .setRatingCol("rating")

print(als)

AlternatingLeastSquare_4182998e70181ae34d9b


In [4]:
model = als.fit(train)
print(model)

AlternatingLeastSquare_4182998e70181ae34d9b


In [5]:
model.write().overwrite().save('/data/books/model.alsmodel')

In [6]:
%autoreload
from mlonspark.alternating_least_square import AlternatingLeastSquareModel

model = AlternatingLeastSquareModel.read().load('/data/books/model.alsmodel')

In [7]:
print(model)

AlternatingLeastSquare_4182998e70181ae34d9b


In [8]:
test = spark.read.format("parquet").load("/data/books/ratings-test.parquet")
test.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)



In [25]:
predictions = model.transform(test)
predictions.printSchema()
predictions.show(10)

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- prediction: float (nullable = false)

+-------+----------+-----------+------+------+--------------------+----+------+------------+
|User-ID|      ISBN|Book-Rating|rating|bookId|            Location| Age|userId|  prediction|
+-------+----------+-----------+------+------+--------------------+----+------+------------+
| 100002|0345397819|          8|   8.0| 23745|dunkirk, new york...|NULL|100002|         NaN|
| 100004|0061015725|          0|   0.0|  2430|san ysidro, calif...|   0|100004|  0.02808103|
| 100004|0345339711|          0|   0.0|   780|san ysidro, calif...|   0|100004|  0.15121098|
| 100004|0446604275|          0|   0.0|  4960|san ysidro, calif...|   0|100004|-0.0092

In [30]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import isnan

evaluator = RegressionEvaluator()\
    .setMetricName("rmse")\
    .setLabelCol("rating")\
    .setPredictionCol("prediction")

predictionsFiltered = predictions.where(~isnan(col("prediction")))
                                
print("number of predictions = %i" % predictionsFiltered.count())

rmse = evaluator.evaluate(predictionsFiltered)

print("RMSE = %f" % rmse)

number of predictions = 279095
RMSE = 4.668829
